# Stack Exchange Question Classifier

Stack Exchange é um repositório de informações que possui 105 tópicos diferentes e cada tópico tem uma acervo de perguntas que foram feitas e respondidas por membros experientes da comunidade do StackExchange. Os tópicos são tão diversos quanto viagens, culinária, programação, engenharia e fotografia. Foram escolhidos 10 categoria de tópicos (gis, security, photo, mathematica, unix, wordpress, scifi, electronics, android, apple) e dada uma pergunta e um trecho, sua tarefa é identificar quais dentre os 10 tópicos a que pertence.

In [149]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

### 1. Leitura e formatação dos dados

In [192]:
data = pd.read_json('training.json', orient='columns')
print data

                                                 excerpt  \
0      I'm trying to work out, in general terms, the ...   
1      Can I know which component senses heat or acts...   
2      I am replacing a wall outlet with a Cooper Wir...   
3      i have been reading about the buck converter, ...   
4      I need help with deciding on a Master's Projec...   
5      Is it possible to supply power to a very high ...   
6      My solenoid is part of an old espresso machine...   
7      I use IP core PCI-E for Virtex-6 v.2.5\n\nTher...   
8      I want to replace an ON Semicondutor BC557 PNP...   
9      I will be using the AT32UC3C2512C, the AVR3276...   
10     So I had an old gen 1 ipad that my son tried t...   
11     I flashed the router with OpenWRT and it worke...   
12     I posted a similar question a while ago and go...   
13     I need to buy a voltage converter(adapter) for...   
14     I had purchased an ATMEGA 16 microcontroller a...   
15     My computer AVR's switch just got

Para executar algoritmos de aprendizado de máquina, precisamos converter os arquivos de texto em vetores de features numéricas. Nós estaremos usando o modelo 'bag of words' para o nosso exemplo. Resumidamente, nós segmentamos cada arquivo de texto em palavras (dividido pelo espaço) e contamos o número de vezes que cada palavra ocorre em cada documento e finalmente atribuímos a cada palavra um ID inteiro. Cada palavra única no nosso dicionário irá corresponder a uma característica 

In [191]:
df = CountVectorizer(stop_words='english', strip_accents='unicode')

data_test = data['excerpt']
y = data['topic']

X_train = df.fit_transform(data_test)
X_train.shape

(20219, 29135)

Selecionando cada tipo de variável (features e target) do conjunto de dados. Apenas contar o número de palavras em cada documento tem um problema: ele dará mais peso a documentos mais longos do que documentos mais curtos. Para evitar isso, podemos usar a frequência (TF - Term Frequencies) e, também, podemos até reduzir o peso de palavras mais comuns (TF-IDF - Term Frequency times inverse document frequency).

In [183]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train)
X_train_tfidf.shape

(20219, 29135)

Existem vários algoritmos que podem ser usados para classificação de texto, vamos usar Naive Bayes do sklearn.

### 2. Treinamento

In [184]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### 3. Teste

In [187]:
#data_test_json = json.load(open('input00.txt'))
data_test_json = pd.read_json('input00.txt', orient='columns')

In [188]:
X = data_test_json['excerpt']
#for line in data_test:
#    test = [line['excerpt']]

X_test = df.transform(X)

In [189]:
y_pred = naive_bayes.predict(X_test)

In [199]:

import pandas as pd

#output = json.load(open('output00.txt'))
y_true = pd.read_table('output00.txt')

success = 0
fail = 0
for x, y in zip(X_test, y_true):
    y_pred = naive_bayes.predict(x)
    if y_pred == y:
        success += 1
    else:
        fail += 1

print 'Success: ', success/(success + fail)

Success:  1
